In [0]:
from fastai.vision import *
from fastai.metrics import error_rate
import pandas as pd
import torch.nn as nn
import cv2
import concurrent
from tqdm import tqdm

In [162]:
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
base = Path('/content/drive/My Drive/images')
train = base/'train/'
occ = base/'people_crops/'
test = base/'test/'

# Enable GPU
Runtime -> Change Runtime Type -> GPU

# Retrieve Dataset

-- Do not need to run this anymore -- 

In [0]:
# images/train/chain_id/hotel_id/data_source/image_id.jpg
def extract(path, limit=False, occlusion_type=None):
    data = []
    i = 0
    for chain_id in tqdm(path.ls()):
        for hotel_id in chain_id.ls():
            for d_source in hotel_id.ls():
                for image_id in d_source.ls():
                    i += 1
                    d_row = [chain_id.name, 
                             hotel_id.name, 
                             d_source.name, 
                             image_id.name,
                             str(image_id)]
                    
                    # test specific check
                    d_row += [occlusion_type] if occlusion_type else []
                    
                    if image_id.name != 'masks':
                        data.append(d_row)
                        
                    if limit and i > limit:
                        return data
    return data

def extract_occlusions(limit=False):
    data, i = [], 0
    for ci in tqdm(c.ls()):
        i += 1
        data.append([ci.name,
                    str(ci)])
        if limit and i > limit:
            return data
    return data

In [0]:
# Define the # of training examples/occlusions
limit = False
data = extract(train, limit=limit)
occ = extract_occlusions(limit=limit)

100%|██████████| 2170/2170 [00:00<00:00, 166233.92it/s]


In [0]:
data_df = pd.DataFrame(data, columns=['chain_id',
                              'hotel_id',
                              'd_source',
                              'image_id',
                              'path'])
occ_df = pd.DataFrame(occ, columns=['occlusion_id', 'path'])

In [0]:
data_df.head()

,chain_id,hotel_id,d_source,image_id,path
0,3,18187,travel_website,1192036.jpg,/content/drive/My Drive/images/train/3/18187/t...
1,3,18187,travel_website,1192035.jpg,/content/drive/My Drive/images/train/3/18187/t...
2,3,18187,travel_website,1192038.jpg,/content/drive/My Drive/images/train/3/18187/t...
3,3,18187,travel_website,1192037.jpg,/content/drive/My Drive/images/train/3/18187/t...
4,3,18187,travel_website,1192040.jpg,/content/drive/My Drive/images/train/3/18187/t...


In [0]:
occ_df.head()

,occlusion_id,path
0,4029.png,/content/drive/My Drive/images/people_crops/40...
1,3746.png,/content/drive/My Drive/images/people_crops/37...
2,3020.png,/content/drive/My Drive/images/people_crops/30...
3,3034.png,/content/drive/My Drive/images/people_crops/30...
4,28494.png,/content/drive/My Drive/images/people_crops/28...


In [0]:
data_df.to_csv('train.csv', index=False)
occ_df.to_csv('occlusions.csv', index=False)

In [0]:
files.download('train.csv')
files.download('occlusions.csv')

# Occlusions
Define occlusion operations

In [0]:
def apply_occlusion(img, occlusion_idx=None, scale_resize=1., max_fill_percent=0.8):
    img_c, img_h, img_w = img.shape
    
    # load occlusion
    # TODO: optimize for faster imread + uniform sampling across crops
    occ_idx = np.random.randint(len(occ_df)) if not occlusion_idx else occlusion_idx
    occ_path = occ_df.loc[occ_idx].path 
    occ = cv2.imread(occ_path)

    # random resize taking up < `max_fill_percent`% of image 
    while True:
        resized_occ = cv2.resize(occ, (0, 0),
                               fx=max(np.random.uniform(low=0.1, high=0.7) * scale_resize, 0.4), 
                               fy=max(np.random.uniform(low=0.1, high=0.7) * scale_resize, 0.4))
        percent_filled = np.sum(resized_occ[:, :, 0] // 255) / (img_h * img_w)
        h, w, _ = resized_occ.shape
    
        if percent_filled < max_fill_percent and h < img_h and w < img_w: 
            occ_mask = (1 - resized_occ // 255)
            break
    
    
    occ_h, occ_w, _ = occ_mask.shape
    d_height = img_h - occ_h 
    d_width = img_w - occ_w
    
    # random placement 
    p_top = np.random.uniform() 
    p_left = np.random.uniform() 
    top_fill = int(d_height * p_top)
    bottom_fill = d_height - top_fill
    left_fill = int(d_width * p_left)
    right_fill = d_width - left_fill
    
    fit_occ_mask = cv2.copyMakeBorder(occ_mask, top_fill, bottom_fill, left_fill, right_fill, cv2.BORDER_CONSTANT)
    tensor_occ_mask = torch.from_numpy(1 - fit_occ_mask[:, :, 0]).type(torch.float)

    return tensor_occ_mask * img    

In [0]:
# apply occlusions to training images
def get_occlusion_trans(max_fill_percent=0.8):
  def _occlusion(x):
      x = apply_occlusion(x, max_fill_percent=max_fill_percent)
      return x
  return TfmPixel(_occlusion)

# Create Databunch

In [0]:
# load data frames for training
data_df = pd.read_csv(base/'train.csv')
occ_df = pd.read_csv(base/'occlusions.csv')

In [0]:
data_df.head()

,chain_id,hotel_id,d_source,image_id,path
0,3,18187,travel_website,1192036.jpg,/content/drive/My Drive/images/train/3/18187/t...
1,3,18187,travel_website,1192035.jpg,/content/drive/My Drive/images/train/3/18187/t...
2,3,18187,travel_website,1192038.jpg,/content/drive/My Drive/images/train/3/18187/t...
3,3,18187,travel_website,1192037.jpg,/content/drive/My Drive/images/train/3/18187/t...
4,3,18187,travel_website,1192040.jpg,/content/drive/My Drive/images/train/3/18187/t...


In [0]:
batch_size = 32
img_size = 224
occ = get_occlusion_trans(max_fill_percent=0.8)

ds_tfms = get_transforms(max_rotate=35, max_zoom=1.4, max_lighting=0.2,
                         max_warp=0.4, p_affine=1.0, p_lighting=1.0,
                         xtra_tfms=[occ()])
label_col = 0 # predict the chain_id


data = ImageDataBunch.from_df('/', 
                              data_df, 
                              label_col=label_col,
                              fn_col=-1, 
                              ds_tfms=ds_tfms, 
                              size=img_size,
                              bs=batch_size)

In [0]:
data.show_batch(rows=2, figsize=(10, 10))

# Training

In [0]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:00<00:00, 113574136.71it/s]


In [0]:
lr_find(learn)
learn.recorder.plot()

In [0]:
lr=2e-3

In [0]:
learn.fit_one_cycle(10, slice(lr))

In [0]:
learn.save('stage-1')

In [0]:
learn.unfreeze()

In [0]:
lrs = slice(lr/100, lr)

In [0]:
learn.fit_one_cycle(12, lrs, pct_start=0.8)

In [0]:
learn.save('stage-2');

# Testing the Model

## Extract Test Data
-- Do not need to run this anymore -- 


In [0]:
test_df = pd.read_csv(base/'test_set.csv')
test_df.head()

,image_id,hotel_id,image_url,image_source,upload_timestamp
0,2587571,79733,https://traffickcam.com/images/2017/4/20160304...,traffickcam,3/4/16 23:35
1,2587572,79733,https://traffickcam.com/images/2017/4/20160304...,traffickcam,3/4/16 23:35
2,2587573,79733,https://traffickcam.com/images/2017/4/20160304...,traffickcam,3/4/16 23:35
3,2587574,79733,https://traffickcam.com/images/2017/4/20160304...,traffickcam,3/4/16 23:35
4,2587796,88725,https://traffickcam.com/images/2017/6/20160531...,traffickcam,5/31/16 19:23


In [0]:
# Get Test Set 
!wget https://www2.seas.gwu.edu/~astylianou/hotels50k/test.tar.gz
!tar xvzf test.tar.gz
!rm test.tar.gz
!mv test/ drive

In [0]:
# Extract test_set
occ_types = ['high_occlusions', 'low_occlusions', 'medium_occlusions', 'unoccluded']
test_data = [extract(Path('test/')/occ, occlusion_type=occ) for occ in occ_types]

In [0]:
# return a tests data frame
def get_dframe(data):
     return pd.DataFrame(data, columns=['chain_id',
                              'hotel_id',
                              'd_source',
                              'image_id',
                              'path', 'occlusion_type'])

In [0]:
test_dfs = [get_dframe(d) for d in test_data]
test_df = pd.concat(test_dfs)

In [0]:
test_df.sample(frac=1).head()

In [0]:
test_df.to_csv('test.csv', index=False)

## Evaluating the Model

In [0]:
test_df = pd.read_csv(base/'test.csv')

In [166]:
test_df.sample(frac=1).head()

,chain_id,hotel_id,d_source,image_id,path,occlusion_type
53,61,49863,traffickcam,3843217.jpg,test/unoccluded/61/49863/traffickcam/3843217.jpg,unoccluded
23,61,100231,traffickcam,3944083.jpg,test/low_occlusions/61/100231/traffickcam/3944...,low_occlusions
62,61,89071,traffickcam,2842177.jpg,test/unoccluded/61/89071/traffickcam/2842177.jpg,unoccluded
20,61,49863,traffickcam,3843220.jpg,test/low_occlusions/61/49863/traffickcam/38432...,low_occlusions
22,61,49863,traffickcam,3843218.jpg,test/low_occlusions/61/49863/traffickcam/38432...,low_occlusions


In [0]:
batch_size = 32
img_size = 224
label_col = 0 # predict the chain_id


test_data = ImageDataBunch.from_df('/content/drive/My Drive/images/', 
                              test_df, 
                              label_col=label_col,
                              fn_col=-2
                              size=img_size,
                              bs=batch_size)

In [0]:
learner_path = 'stage-2'

In [0]:
# load trained model for predictions
learn = load_learner(learn_path, test=test_data)
preds, _ = learn.get_preds(ds_type=DatasetType.Test)